**라이브러리 설치 및 데이터 로드**

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install transformers torch scikit-learn # sklearn이라고 하면 오류 발생

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# 데이터 로드
file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/lawsuit_type.csv'  # 데이터셋 경로를 지정하세요.
data = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
print(data.head())

                                            Question Answer  Label
0  중국에서 사채업을 하던 중국 국적의 갑이 대한민국에서 영업을 하기 위하여 대한민국에...     민사      0
1  갑 지방자치단체가 을 주식회사 소유의 토지가 포함된 부지에 대하여 제2종 지구단위계...     민사      0
2  지적을 제곱미터 단위까지만 표시하도록 한 지적법    같은법시행령  의 취지 및 1...     민사      0
3             전득자가 전매자를 대위하여 처분금지가처분결정을 받은 경우 그 효력범위     민사      0
4     근로자성 인정을 위한 사용종속관계 유무의 판단에 있어 그 고려 요소들의 입체적 파악     민사      0


**데이터 전처리 및 BERT 모델 설정**

In [ ]:
# 데이터 전처리
# 필요한 경우 추가 데이터 전처리를 수행하세요.

# BERT 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # num_labels는 클래스의 개수에 따라 조정하세요.

# 데이터셋 클래스 정의
class LawsuitDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_token_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item_index):
        text = self.texts[item_index]
        label = self.labels[item_index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 준비
X_train, X_val, y_train, y_val = train_test_split(data['Question'], data['Label'], test_size=0.1, random_state=42)
train_dataset = LawsuitDataset(X_train.tolist(), y_train.tolist(), tokenizer)
val_dataset = LawsuitDataset(X_val.tolist(), y_val.tolist(), tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**학습과 평가를 위한 Trainer 설정 및 모델 학습**

In [ ]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# 트레이너 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

**테스트 데이터에 대한 예측 수행**

In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs.argmax().item()

new_text = "우리 회사 어떤 직원이 나와의 계약을 어겼어."
print(predict(new_text))

0


In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs.argmax().item()

new_text = "내 친구가 사람을 죽였어."
print(predict(new_text))

0
